# MNIST Data Set 1-Channel to 3-Channel

In [14]:
# -------------------------------------------------------------------------------------------------
# ----------Imports--------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------

# --> Basic Imports

import numpy as np
import matplotlib.pyplot as plt
# import seaborn_image as isns
import pickle 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset

# --> Torch imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.cuda import device_count, current_device
from torch.backends.cudnn import version
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets, transforms
import torch.nn.functional as fct
import torchvision

# --> System imports
import sys
import datetime
# import os
# import ti

In [9]:
torch.set_default_tensor_type(torch.FloatTensor)
#---------------------------------------------------------------------
# Digits MNIST Data
MNIST_training_batch_size = 600
MNIST_test_batch_size = 10000


MNIST_full_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [1])
            ])),
        batch_size=60000, shuffle=True, num_workers=0)#4

MNIST_train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [1])
            ])),
        batch_size=MNIST_training_batch_size, shuffle=True, num_workers=0)#4

MNIST_test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [1])
            ])),
        batch_size=MNIST_test_batch_size, shuffle=True)

In [10]:
def convert_rgb(images, colors):
    colors = colors.tolist()
    idx = 0
    for image in images:
        color = colors[idx]
        if color == 0:
            image[0, :, :][image[0, :, :] != -0.5] = 1
            image[0, :, :][image[0, :, :] == -0.5] = 0
            image[1, :, :] = 0
            image[2, :, :] = 0
        elif color == 1:
            image[0, :, :] = 0
            image[1, :, :][image[1, :, :] != -0.5] = 1
            image[1, :, :][image[1, :, :] == -0.5] = 0
            image[2, :, :] = 0
        elif color == 2:
            image[0, :, :] = 0
            image[1, :, :] = 0
            image[2, :, :][image[2, :, :] != -0.5] = 1
            image[2, :, :][image[2, :, :] == -0.5] = 0

        idx += 1
    return images

In [11]:
# Load the complete training data to
MNIST_train_data = next(iter(MNIST_full_loader))
len(MNIST_train_data)
# Converting the 1-channel images data to 3-channel images
MNIST_train_1channel = MNIST_train_data[0]
MNIST_train_data[0]= torch.repeat_interleave(MNIST_train_data[0], 3, dim= -3)
# Generating random categorical variables from 1,2,3 as R,G,B
MNIST_color_labels = torch.randint(0,3,(1,60000)).view(60000)
#color_labels = torch.randint(1,4,(1,60000)).tolist()[0]
#full_data.append(torch.Tensor(color_labels))
MNIST_train_data.append(MNIST_color_labels)
MNIST_labels30 = MNIST_color_labels*10 + MNIST_train_data[1]
# Convert the 3- Channel images to 3-channel RGB images
MNIST_train_data[0] = convert_rgb(MNIST_train_data[0], MNIST_train_data[2])
MNIST_train_data.append(MNIST_train_1channel)
MNIST_train_data.append(MNIST_labels30)
print(len(MNIST_train_data))
print("MNIST_train_data shape: ",MNIST_train_data[0][0].shape)
print("MNIST_train_data labels shape: ",MNIST_train_data[1].shape)
print("MNIST_train_data colors shape: ",MNIST_train_data[2].shape)

# 3-channel images, num_labels, col_labels, 1-channel images
MNIST_train_dataset =TensorDataset(MNIST_train_data[0], MNIST_train_data[1], MNIST_train_data[2], MNIST_train_data[3], MNIST_train_data[4])
MNIST_train_dataloader = DataLoader(MNIST_train_dataset, batch_size=600, shuffle = True)

# Load the complete Testing data to
MNIST_test_data = next(iter(MNIST_test_loader))
len(MNIST_test_data)
# Converting the 1-channel images data to 3-channel images
MNIST_test_1channel = MNIST_test_data[0]
# Converting the 1-channel images data to 3-channel images
MNIST_test_data[0]= torch.repeat_interleave(MNIST_test_data[0], 3, dim= -3)
# Generating random categorical variables from 1,2,3 as R,G,B
MNIST_test_color_labels = torch.randint(0,3,(1,10000)).view(10000)
#test_color_labels = torch.randint(1,4,(1,60000)).tolist()[0]
#test_data.append(torch.Tensor(test_color_labels))
MNIST_test_data.append(MNIST_test_color_labels)
MNIST_test_data.append(MNIST_test_1channel)
MNIST_labels30 = MNIST_test_color_labels*10 + MNIST_test_data[1]
MNIST_test_data.append(MNIST_labels30)
# Convert the 3- Channel images to 3-channel RGB images
MNIST_test_data[0] = convert_rgb(MNIST_test_data[0], MNIST_test_data[2])
print(len(MNIST_test_data))
print("MNIST_test_data shape: ",MNIST_test_data[0][0].shape)
print("MNIST_test_data labels shape: ",MNIST_test_data[1].shape)
print("MNIST_test_data colors shape: ",MNIST_test_data[2].shape)

# 3-channel images, num_labels, col_labels, 1-channel images
MNIST_test_dataset = TensorDataset(MNIST_test_data[0], MNIST_test_data[1], MNIST_test_data[2], MNIST_test_data[3], MNIST_test_data[4])
MNIST_test_dataloader = DataLoader(MNIST_test_dataset, batch_size=MNIST_test_batch_size, shuffle = True)

5
MNIST_train_data shape:  torch.Size([3, 28, 28])
MNIST_train_data labels shape:  torch.Size([60000])
MNIST_train_data colors shape:  torch.Size([60000])
5
MNIST_test_data shape:  torch.Size([3, 28, 28])
MNIST_test_data labels shape:  torch.Size([10000])
MNIST_test_data colors shape:  torch.Size([10000])


In [12]:
mnistdigits_training_data = {"images_rgb":MNIST_train_data[0], 
                             "images_gray":MNIST_train_data[3],
                             "digits":MNIST_train_data[1], 
                             "colors":MNIST_train_data[2], 
                             "total_categories":MNIST_train_data[4]
                             }
mnistdigits_testing_data = {"images_rgb":MNIST_test_data[0], 
                             "images_gray":MNIST_test_data[3],
                             "digits":MNIST_test_data[1], 
                             "colors":MNIST_test_data[2], 
                             "total_categories":MNIST_test_data[4]
                             }

In [15]:
with open('mnistdigits_training_data_v3_test.pkl', 'wb') as f:
    pickle.dump(mnistdigits_training_data, f)
    
with open('mnistdigits_testing_data_v3_test.pkl', 'wb') as f:
    pickle.dump(mnistdigits_testing_data, f)